# Cleaning Trips (DO) Data

### Part 1 of GPS Headway Processing

<b>Input:</b>
- DO Reguler Transit
- DO BKO Transit
- Google Form for Specified Time
- List of Active Trips
- Stops Sequence

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import time
import requests
#pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 120)
# import haversine as hs
# import geopandas as gpd
# import pyproj
import os
import math

### 1. Open and Merge DO Data

#### 1. DO Transit

a. DO Reguler Transit

In [119]:
dfrt = pd.read_excel('../Data Transit Raw/September/all/Administrator  Transjakarta Transit_22_Sep.xlsx', header=1,
                    dtype= {'No Do': str, 'Rute Real': str})

In [120]:
dfrt.head(2)

,Operator,No Bus,Nik,Pramudi,SPBU,Rute Real,Rute Renops,Rute Status,KM Start,KM End,KM Baku,Kategori DO,No Do,Tanggal Do,Jam Start Do,Jam End Do,Respons Verifikator,Respons Admin
0,BayuHolongPersada SB 31,BHL 220512,2205045,ULLI AMRI,NaN,TMII - TEGAL PARANG - TMII VIA CAWANG SUTOYO (7D),TMII - TEGAL PARANG - TMII VIA CAWANG SUTOYO (7D),IncompleteLog0 / 44 Halte,112993,113026,NaN,DO Reguler,202309222059181255,2023-09-22,20:59,22:16,Proses,Proses
1,BayuHolongPersada SB 31,BHL 220508,2205049,Pepep Sopyan,NaN,TMII - TEGAL PARANG - TMII VIA CAWANG SUTOYO (7D),TMII - TEGAL PARANG - TMII VIA CAWANG SUTOYO (7D),IncompleteLog0 / 44 Halte,115089,115121,NaN,DO Reguler,202309222044481084,2023-09-22,20:45,22:05,Proses,Proses


In [121]:
dfrt['Tanggal Do'].unique()

array(['2023-09-22'], dtype=object)

In [5]:
# dfrt= dfrt[(dfrt['Tanggal Do']=='2023-04-07') | (dfrt['Tanggal Do']=='2023-04-08') | 
#           (dfrt['Tanggal Do']=='2023-04-09')]

b. DO BKO Transit

In [122]:
dfbko = pd.read_excel('../Data Transit Raw/September/all/Administrator  Transjakarta Transit BKO_22 Sep.xlsx', header=1,
                     dtype= {'No Do': str, 'Rute Real': str})

In [7]:
dfbko.head(2)

,Operator,No Bus,Nik,Pramudi,SPBU,Rute Real,Rute Renops,Rute Status,KM Start,KM End,KM Baku,Kategori DO,No Do,Tanggal Do,Jam Start Do,Jam End Do,Respons Verifikator,Respons Admin
0,Bianglala Metropolitan (BMP AMARI),NaN,200065,DEDE RUSMADI,NaN,KALI BESAR BARAT - BLOK M (AMARI DAMPAK MRT 2A...,LEBAK BULUS - PASAR BARU (AMARI) (M8),IncompleteLog0 / 26 Halte,244324,NaN,NaN,DO BKO,202309220839231705,2023-09-22,22:30,NaN,Proses,Proses
1,BIANGLALA METROPOLITAN (BMP) SB 58,BMP 283,221014,FAHRI,NaN,DUKUH ATAS 2 - RAGUNAN (6),RAGUNAN - MH THAMRIN VIA SEMANGGI (END IRTI) (6B),IncompleteLog0 / 19 Halte,64275,64275,NaN,DO BKO,202309221950471486,2023-09-22,20:13,20:13,Proses,Proses


In [8]:
dft = pd.concat([dfrt, dfbko])
len(dft)

10840

46

3. DO Manual

a. BRT

In [66]:
dfmb = pd.read_excel('../Data Transit Raw/DO Manual/20230927_DO Manual BRT (Responses).xlsx', dtype={'Tanggal Do': str, 'Jam Start Do': str})
# dfmb['Day'] = dfmb['Tanggal Do'].apply(lambda x: x.day)
# dfmb['Month'] = dfmb['Tanggal Do'].apply(lambda x: x.month)
# dfmb = dfmb[dfmb['Month']==4]
# dfmb = dfmb[(dfmb['Day']==7) | (dfmb['Day']==8) | (dfmb['Day']==9) ]
# dfmb = dfmb[list(dft.columns)]
dfmb['No Do']=dfmb['Nik'].apply(lambda x: 'MANUAL'+str(x)+ str(np.random.randint(low=0, high=100)))

In [67]:
dfmb.head(2)

,Operator,No Bus,Nik,Pramudi,SPBU,Rute Real,Rute Renops,Rute Status,KM Start,KM End,KM Baku,Kategori DO,No Do,Tanggal Do,Jam Start Do,Jam End Do,Respons Verifikator,Respons Admin,Timestamp,NIK Petugas,KODE RUTE,Alasan tidak menggunakan aplikasi transit,"Apabila trip yang digunakan tidak tersedia di pilihan di atas, masukan nama trip di sini:",Trip yang digunakan:
0,NaN,BMP 231,230632,NaN,NaN,PLUIT - PINANG RANTI (M9),NaN,NaN,NaN,NaN,NaN,NaN,MANUAL23063273,2023-09-22 00:00:00,00:30:00,NaN,NaN,NaN,2023-09-22 00:02:48.647,102067,M9,Belum Serah Terima Kunci,NaN,PLUIT - PINANG RANTI (M9)
1,NaN,BMP 231,230632,NaN,NaN,PLUIT - PINANG RANTI (M9),NaN,NaN,NaN,NaN,NaN,NaN,MANUAL23063263,2023-09-22 00:00:00,04:00:00,NaN,NaN,NaN,2023-09-22 03:48:10.891,102067,M9,Belum Serah Terima Kunci,NaN,PLUIT - PINANG RANTI (M9)


b. N-BRT

In [68]:
dfmnb = pd.read_excel('../Data Transit Raw/DO Manual/20230922_DO Manual Non-BRT (Responses).xlsx', 
                      dtype={'Tanggal Do': str, 'Jam Start Do': str})
# dfmnb['Day'] = dfmnb['Tanggal Do'].apply(lambda x: x.day)
# dfmnb['Month'] = dfmnb['Tanggal Do'].apply(lambda x: x.month)
# dfmnb = dfmnb[dfmnb['Month']==4]
# dfmnb = dfmnb[(dfmnb['Day']==7) | (dfmnb['Day']==8) | (dfmnb['Day']==9)]
# dfmnb = dfmnb[list(dft.columns)]
dfmnb['No Do']=dfmnb['Nik'].apply(lambda x: 'MANUAL'+str(x)+ str(np.random.randint(low=0, high=100)))

In [64]:
dfmnb.head(1)

,Operator,No Bus,Nik,Pramudi,SPBU,Rute Real,Rute Renops,Rute Status,KM Start,KM End,KM Baku,Kategori DO,No Do,Tanggal Do,Jam Start Do,Jam End Do,Respons Verifikator,Respons Admin,Timestamp,NIK Petugas,KODE RUTE,Alasan tidak menggunakan aplikasi transit,"Apabila trip yang digunakan tidak tersedia di pilihan di atas, masukan nama trip di sini:",Trip yang digunakan:,Unnamed: 24
0,NaN,MYS 18180,20122098,NaN,NaN,BALAI KOTA - PANTAI MAJU (1A),NaN,NaN,NaN,NaN,NaN,NaN,MANUAL2012209884,2023-09-22 00:00:00,05:15:00,NaN,NaN,NaN,2023-09-22 05:12:06.718,101996,1A,Shift tidak sesuai di aplikasi transit,NaN,NaN,BALAI KOTA - PANTAI MAJU (1A)


c. DO Manual Merged 

In [28]:
# dfmb

In [69]:
dfm = pd.concat([dfmb, dfmnb])
# dfm.rename(columns={'No Bus (lengkap dan dengan spasi)': 'No Bus'}, inplace=True)
# dfm.rename(columns={'Tanggal Start Do': 'Tanggal Do'}, inplace=True)
# # dfm = dfmb.copy()

In [70]:
dfm.dropna(subset='Rute Real', inplace=True)
dfm['Rute Real'] = dfm['Rute Real'].apply(lambda x: x.upper().strip()) 

In [71]:
dfm = dfm[~dfm['Rute Real'].str.contains('POOL')]
dfm = dfm[~dfm['Rute Real'].str.contains('DEPO')]
dfm = dfm[~dfm['Rute Real'].str.contains('PULANG')]
#dfm = dfm.drop(1949)

In [59]:
dfm.head(1)

,Operator,No Bus,Nik,Pramudi,SPBU,Rute Real,Rute Renops,Rute Status,KM Start,KM End,KM Baku,Kategori DO,No Do,Tanggal Do,Jam Start Do,Jam End Do,Respons Verifikator,Respons Admin,Timestamp,NIK Petugas,KODE RUTE,Alasan tidak menggunakan aplikasi transit,"Apabila trip yang digunakan tidak tersedia di pilihan di atas, masukan nama trip di sini:",Trip yang digunakan:,Unnamed: 24
0,NaN,BMP 231,230632,NaN,NaN,PLUIT - PINANG RANTI (M9),NaN,NaN,NaN,NaN,NaN,NaN,MANUAL23063298,2023-09-22,00:30:00,NaN,NaN,NaN,2023-09-22 00:02:48.647,102067,M9,Belum Serah Terima Kunci,NaN,PLUIT - PINANG RANTI (M9),NaN


In [72]:
dfm['No Bus'] = dfm['No Bus'].str.upper()
dfm['No Bus'] = dfm['No Bus'].str.strip()

In [74]:
dfm['Tanggal Do'] = dfm['Tanggal Do'].apply(lambda x : str(x)[0:10])
dfm['Tanggal Do'].unique()

array(['2023-09-22'], dtype=object)

In [75]:
dfm['Jam Start Do'] = dfm['Jam Start Do'].apply(lambda x : str(x)[0:5])
dfm['Jam Start Do'].unique()

array(['00:30', '04:00', '06:15', '06:53', '07:15', '07:55', '11:27',
       '12:35', '13:05', '15:30', '15:40', '16:25', '15:45', '15:42',
       '15:50', '15:55', '16:48', '16:20', '17:00', '17:26', '17:37',
       '16:42', '17:47', '19:00', '19:10', '20:00', '21:22', '22:00',
       '22:06', '22:10', '22:15', '22:20', '22:30', '22:40', '22:35',
       '23:00', '23:06', '05:15', '06:30', '06:48', '06:50', '06:35',
       '08:00', '08:45', '11:30', '13:40', '14:25', '14:30', '16:35',
       '16:50', '21:55'], dtype=object)

3. DO Merged

In [123]:
df = pd.concat([dft, dfm])

In [124]:
# dft['Jam Start Do'].unique()
dft['Tanggal Do'].unique()

array(['2023-09-22'], dtype=object)

### 2. Cleaning DO Data

In [125]:
## Convert into time format

dfhari = pd.DataFrame(list(zip([0,1,2,3,4,5,6], ['Senin', 'Selasa', 'Rabu', 'Kamis', 'Jumat', 'Sabtu', 'Minggu' ])),
               columns =['Angka', 'Hari'])


df['TimeStart'] = df['Tanggal Do'] +' '+ df['Jam Start Do']  
df['TimeStart']=df['TimeStart'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M')) 

df['Jam Start Do'] = df['Jam Start Do'].apply(lambda x: datetime.strptime(x, '%H:%M')) 
df['Tanggal Start Do']= df['Tanggal Do'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

df['Weekday'] = df['Tanggal Start Do'].apply(lambda x: 'Weekend' if x.weekday()>4 else 'Weekday')
df['Hari'] = df['Tanggal Start Do'].apply(lambda x: dfhari['Hari'].loc[x.weekday()])

df['Month'] = df['TimeStart'].apply(lambda x:x.month)
df['Week/year'] = df['TimeStart'].apply(lambda x:x.week)



In [126]:
# Informasi tentang trip

df['Kode Rute Real'] = df['Rute Real'].apply(lambda x: x.split('(')[-1][:-1])
df['Looping'] = df['Rute Real'].apply(lambda x: 'Looping' if len(x.split('-'))>2 else 'One Trip')

df['Origin'] = df['Rute Real'].apply(lambda x: x.split('-')[0])
df['Destination'] = df['Rute Real'].apply(lambda x: x.split(' - ')[-1].split(' (')[0].split(' via')[0].split(' VIA')[0])

df['Hour'] = df['TimeStart'].apply(lambda x: x.hour)

In [127]:
# Jenis Layanan

dlayanan = pd.read_excel('../Data Untuk TT/Jenis Layanan.xlsx')
dlayanan = dlayanan[['KODE', 'KATEGORI RUTE']]
dlayanan['Kode Rute Real'] = dlayanan['KODE'].apply(lambda x: str(x))
dlayanan.drop(['KODE'], inplace=True, axis=1)
dlayanan.dropna(subset=['KATEGORI RUTE'], inplace=True)
dlayanan['KATEGORI RUTE'] = dlayanan['KATEGORI RUTE'].apply(lambda x: x.upper()) 

df2 = df.merge(dlayanan, on='Kode Rute Real', how='left')
df2['Jenis Layanan'] = df2['KATEGORI RUTE']
df2.drop('KATEGORI RUTE', axis=1, inplace=True)


In [128]:
df2['Day'] = df2['TimeStart'].apply(lambda x: x.day)

def katwak(Hour):
    if (Hour>3) and (Hour<6) :
        return 'Off Peak Pagi'
    elif (Hour>5) and (Hour<9): #6.00 - 08.59 peak pagi
        return 'Peak Pagi'
    elif (Hour>8) and (Hour<16): # 09.00 - 15.59 off peak siang
        return 'Off Peak Siang'
    elif (Hour>15) and (Hour<20): # 16.00 - 19.59 peak sore 
        return 'Peak Sore'
    elif (Hour>19) and (Hour<23): #20:00 - 22.59 off peak malam 
        return 'Off Peak Malam'
    elif (Hour>22):
        return 'Luar Jam Operasi Reguler'
    elif (Hour<4):
        return 'Luar Jam Operasi Reguler'
    
df2['Kategori Waktu'] = df2.apply(lambda x: katwak(x['Hour']), axis=1)

In [129]:
df2.loc[df2['No Bus'].isna()==False, 'Operator'] =  df2.loc[df2['No Bus'].isna()==False]['No Bus'].apply(lambda x: x[0:3])

df2.loc[(df2['Operator'].isna()==False) & (df2['Operator'].str.contains('TJ')), 'Operator'] = 'TJ'
df2.loc[(df2['Operator'].isna()==False) & (df2['Operator'].str.contains('MB')), 'Operator'] = 'MB'

In [130]:
df2.loc[df2['No Bus'].isna()==False, 'No Bus'] = df2.loc[df2['No Bus'].isna()==False]['No Bus'].apply(lambda x: x.replace(' ', '').upper())

In [131]:
### Fill End DO -> The duration might be too long and hence does not make sense but it is okay 
### because we only need it to limit the GPS's time range for each trip

In [132]:
len(dfx['No Bus'])

10902

In [133]:
dfx = df2.copy()

dfx.loc[dfx['Jam End Do'].isna() == False, 'TanpaEndDo'] = 0
dfx.loc[dfx['Jam End Do'].isna() == True, 'TanpaEndDo'] = 1

dfx.sort_values(by=['No Bus', 'TimeStart'], ascending=True, inplace=True)
dfx.reset_index(drop=True, inplace=True)

dfx['End DO New'] = dfx['Jam End Do']
inoend = dfx[dfx['TanpaEndDo']==1].index

listnoend = []
for i in inoend:
    if i+1 < len(dfx)-1:
        if dfx['No Bus'].loc[i] == dfx['No Bus'].loc[i+1]:
            if dfx['Tanggal Start Do'].loc[i] == dfx['Tanggal Start Do'].loc[i+1]:
                listnoend.append([i, dfx['Jam Start Do'].loc[i+1]])
            else: 
                listnoend.append([i, np.nan])
        else:
            listnoend.append([i, np.nan])
        
dfnoend =pd.DataFrame(listnoend, columns = ['Index', 'End DO New'])
dfnoend.set_index(['Index'], inplace=True)

for i in list(dfnoend.index):
    dfx['End DO New'].loc[i] = str(dfnoend['End DO New'].loc[i])[11:16]
    
dfx.loc[dfx['End DO New']=='', 'End DO New'] = np.nan

dfx['Tanggal End DO New'] = dfx['Tanggal Start Do']

C:\Users\Nadya\AppData\Local\Temp\ipykernel_15796\3845528507.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx['End DO New'].loc[i] = str(dfnoend['End DO New'].loc[i])[11:16]


In [134]:
len(dfx[(dfx['TanpaEndDo']==1) & (dfx['End DO New'].isna())])

61

In [135]:
df = dfx.copy()

In [136]:
df['Jenis Layanan'].unique()

array(['ANGKUTAN UMUM INTEGRASI', 'BRT', 'BRT AMARI', 'TRANSJABODETABEK',
       nan, 'RUSUN', 'ROYALTRANS'], dtype=object)

In [137]:
## Adding Travel Time 

df = dfx.copy()

df['JamStart2'] = df['Jam Start Do'].apply (lambda x: str(x)[11:16])
df.loc[(df['Jenis Layanan']!='BRT AMARI') & (df['Jam Start Do'].apply(lambda x: str(x)[11:13]=='04')), 'JamStart2'] = '05:00'


df['Tanggal Start2'] = df['Tanggal Start Do'].apply (lambda x: str(x)[:11])
df['TimeStart2'] = df['Tanggal Start2'] +' '+ df['JamStart2']  
df['TimeStart2']=df['TimeStart2'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M')) 

df.drop('JamStart2', axis=1, inplace=True)


dfnoend = df.copy()
dfnoend = dfnoend[dfnoend['End DO New'].isna()]
dfnoend['TimeEnd'] = [np.nan]*len(dfnoend)
dfnoend['TTMinutes'] = [np.nan]*len(dfnoend)

df = df.dropna(subset = ['End DO New'])

df['TimeEnd'] =  df['Tanggal Start2' ]+' '+ df['End DO New'].apply(lambda z: str(z))
df['TimeEnd']=df['TimeEnd'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M')) 

df['TTDays'] = (df['TimeEnd'] - df['TimeStart2'])
df['TTMinutes'] = df['TTDays'].apply(lambda x: x.seconds/60)

df = df.drop('TTDays', axis=1)
df.drop('Tanggal Start2', axis=1, inplace=True)
df = pd.concat([df, dfnoend])



In [138]:
df.head(1)

,Operator,No Bus,Nik,Pramudi,SPBU,Rute Real,Rute Renops,Rute Status,KM Start,KM End,KM Baku,Kategori DO,No Do,Tanggal Do,Jam Start Do,Jam End Do,Respons Verifikator,Respons Admin,Timestamp,NIK Petugas,KODE RUTE,Alasan tidak menggunakan aplikasi transit,"Apabila trip yang digunakan tidak tersedia di pilihan di atas, masukan nama trip di sini:",Trip yang digunakan:,Unnamed: 24,TimeStart,Tanggal Start Do,Weekday,Hari,Month,Week/year,Kode Rute Real,Looping,Origin,Destination,Hour,Jenis Layanan,Day,Kategori Waktu,TanpaEndDo,End DO New,Tanggal End DO New,TimeStart2,TimeEnd,TTMinutes,Tanggal Start2
0,BHL,BHL220508,2205035,JONNI MARASIH SIREGAR,NaN,TMII - TEGAL PARANG - TMII VIA CAWANG SUTOYO (7D),TMII - TEGAL PARANG - TMII VIA CAWANG SUTOYO (7D),IncompleteLog0 / 44 Halte,114895,114931,NaN,DO Reguler,202309220558051375,2023-09-22,1900-01-01 05:59:00,07:59,Proses,Proses,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2023-09-22 05:59:00,2023-09-22,Weekday,Jumat,9,38,7D,Looping,TMII,TMII,5,ANGKUTAN UMUM INTEGRASI,22,Off Peak Pagi,0.0,07:59,2023-09-22,2023-09-22 05:59:00,2023-09-22 07:59:00,120.0,NaN


In [139]:
# df['No Bus']

In [140]:
def cleantripdata (df, timestart, timeend, extduration):
    
    #df['TimeStart'] = df['TimeStart'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    
    # Selecting only the trips within the time we want
    df['InsideGPStime'] = [0] *len(df)
    df.loc[(df['TimeStart'].apply(lambda x: x.time()) > datetime.strptime(timestart, '%H:%M').time()) & (df['TimeStart'].apply(lambda x: x.time()) < datetime.strptime(timeend, '%H:%M').time()),  'InsideGPStime']= 1
    df = df[df['InsideGPStime']==1]    
    
    # Deleting the ones which travel duration less than our 'error trip' threshold
    df = df[(df['TTMinutes'].isna()) | (df['TTMinutes']>extduration)]
    
    # Clean the Bus Code
    df.dropna(subset=['No Bus'], inplace=True)
    df['No Bus'] = df['No Bus'].apply(lambda x: x.replace(' ','').upper())
    df['No Bus'] = df['No Bus'].apply(lambda x: x.strip())
    
    # Bus Code for TJ and PPD needs to be adjusted to fit the formatting in the GPS data
    df.loc[(df['Operator'] == 'TJ') | (df['Operator'] == 'PPD'), 'No Bus']= df[(df['Operator'] == 'TJ') | (df['Operator'] == 'PPD')]['No Bus'].apply(lambda x:  x[:-3] + '0' +x[-3:] if sum(a.isdigit() for a in x)==3 else x)
    
    # Extra cleaning!
    df.rename(columns={'Unnamed: 0': 'No'}, inplace=True)
    df['No'] = list(range(len(df)))
    df= df[['No', 'No Do', 'No Bus', 'Kode Rute Real', 'Tanggal Do', 'TimeStart', 'TimeEnd', 'Rute Real', 'TTMinutes' ]]

    return df 

In [141]:
dftrip = cleantripdata(df, '04:20', '22:30', 15 )

In [142]:
print(len(dftrip), len(df))
dftrip.head(2)

10723 10902


,No,No Do,No Bus,Kode Rute Real,Tanggal Do,TimeStart,TimeEnd,Rute Real,TTMinutes
0,0,202309220558051375,BHL220508,7D,2023-09-22,2023-09-22 05:59:00,2023-09-22 07:59:00,TMII - TEGAL PARANG - TMII VIA CAWANG SUTOYO (7D),120.0
1,1,202309220759311148,BHL220508,7D,2023-09-22,2023-09-22 08:00:00,2023-09-22 10:46:00,TMII - TEGAL PARANG - TMII VIA CAWANG SUTOYO (7D),166.0


In [143]:
dftrip['Rute Real']= dftrip['Rute Real'].apply(lambda x: x.upper().strip())

In [144]:
dftrip['Rute Real'] = dftrip['Rute Real'].apply(lambda x: x.split(' (')[0].strip().upper())

In [145]:
dftrip['Rute Real']= dftrip['Rute Real'].apply(lambda x: x.upper().strip())

In [146]:
dftrip.loc[(dftrip['Kode Rute Real']=='1'), 'Rute Real'] = dftrip.loc[(dftrip['Kode Rute Real']=='1')]['Rute Real'].str.replace('KOTA', 'KALI BESAR BARAT')

#### Fill Empty End DO

In [147]:
dftrip.sort_values(by=['No Bus', 'TimeStart'], ascending=True, inplace=True)
dftrip['No']= range(len(dftrip))
dftrip.set_index('No', inplace=True)

In [148]:

inoend = dftrip[dftrip['TimeEnd'].isna()].index

listnoend = []
for i in inoend:
    if dftrip['No Bus'].loc[i] == dftrip['No Bus'].loc[i+1]:
        if dftrip['Tanggal Do'].loc[i] == dftrip['Tanggal Do'].loc[i+1]:
            listnoend.append([i, dftrip['TimeStart'].loc[i+1]])
        else: 
            listnoend.append([i, np.nan])
    else:
        listnoend.append([i, np.nan])
        


In [149]:
dfnoend =pd.DataFrame(listnoend, columns = ['Index', 'End DO New'])
dfnoend.set_index(['Index'], inplace=True)
dfnoend = dfnoend[dfnoend['End DO New'].isna()==False]

dfnoend

for i in list(dfnoend.index):
    dftrip['TimeEnd'].loc[i] = str(dfnoend['End DO New'].loc[i])[11:16]
    
dftrip.loc[dftrip['TimeEnd']=='', 'TimeEnd'] = np.nan

C:\Users\Nadya\AppData\Local\Temp\ipykernel_15796\237819773.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftrip['TimeEnd'].loc[i] = str(dfnoend['End DO New'].loc[i])[11:16]


#### Fill End Do (+3 hours)

In [150]:
# pd.to_timedelta(180,'m')
from datetime import timedelta

dftrip.loc[dftrip['TimeEnd'].isna(), 'TimeEnd'] = dftrip[dftrip['TimeEnd'].isna()]['TimeStart'].apply(lambda x: x + pd.to_timedelta(180,'m'))

In [151]:
dftrip[dftrip['TimeEnd'].isna()]

,No Do,No Bus,Kode Rute Real,Tanggal Do,TimeStart,TimeEnd,Rute Real,TTMinutes
No,,,,,,,,


Export processed DO Data

In [152]:
dftrip.to_excel('../Data Transit Raw/September/Processed_all_DO_22 September.xlsx')

In [153]:
dftrip[dftrip['No Bus'].isna()]

,No Do,No Bus,Kode Rute Real,Tanggal Do,TimeStart,TimeEnd,Rute Real,TTMinutes
No,,,,,,,,


### 3. Merge with Stops Sequence

#### Open Stops Sequence

In [217]:
stopseq0 = pd.read_csv('../../Basis Data Alat Produksi/20230821_Basis Data Alat Produksi - Stop Seq-updated.csv', dtype={'Kode Trip': str}, delimiter=';')
# stopseq0 = pd.read_excel('../../Stops Sequence/20230821_Basis Data Alat Produksi - Stop Seq.xlsx', dtype={'Kode Trip': str})

In [218]:
stopseq0[stopseq0['Nama Trip'].isna()]

,ID,Kode Rute,Nama Rute,Kode Trip,Nama Trip,Stop ID,Nama Perhentian,Koordinat,Latitude,Longitude,Urutan,Kode PP,Keterangan,Tipe Perhentian,Kategori Rute,Utama / Non,STATUS OPERASI RUTE,STATUS OPERASI HALTE,STATUS OPERASI TRIP GTFS,STATUS OPERASI TRIP TRANSIT,Unnamed: 20


In [219]:
len(stopseq0)

38384

In [220]:
# stopseq0['Kode Rute'].unique()

In [221]:
stopseq0['Nama Rute'] = stopseq0['Nama Rute'].apply(lambda x: x.upper().strip())
stopseq0['Nama Trip'] = stopseq0['Nama Trip'].apply(lambda x: x.upper().strip())
stopseq0['Kode Trip'] = stopseq0['Kode Trip'].apply(lambda x: x.upper().strip())
stopseq0['Nama Perhentian'] = stopseq0['Nama Perhentian'].apply(lambda x: x.upper().strip())
stopseq0['Nama Trip'] = stopseq0['Nama Trip'].apply(lambda x: x.replace('  ', ' '))
stopseq0 = stopseq0[stopseq0['Kode Rute'].str.contains('JAK')==False]
stopseq0['Nama Trip'] = stopseq0['Nama Trip'].apply(lambda x: x.replace(' - ', '-'))
dftrip['Rute Real'] = dftrip['Rute Real'].apply(lambda x: x.replace(' - ', '-'))
dftrip['Rute Real'] = dftrip['Rute Real'].apply(lambda x: x.replace(' -', '-'))
dftrip['Rute Real'] = dftrip['Rute Real'].apply(lambda x: x.replace('- ', '-'))
stopseq0['Nama Trip'] = stopseq0['Nama Trip'].apply(lambda x: x.replace(' -', '-'))
stopseq0['Nama Trip'] = stopseq0['Nama Trip'].apply(lambda x: x.replace('- ', '-'))

In [222]:
# stopseq = stopseq0[stopseq0['STATUS OPERASI TRIP TRANSIT']=='Beroperasi']

In [223]:
stopseq0['split'] = stopseq0['Nama Trip'].apply(lambda x: len(x.split('-')))
stopseq0['split'].unique()

array([2, 3, 4, 1], dtype=int64)

In [224]:
stopseq0.loc[stopseq0['split']>2, 'Looping'] = 'Looping'
stopseq0.loc[stopseq0['split']<=2, 'Looping'] = 'One Trip'

In [225]:
# stopseq0[stopseq0['split']==3]['Nama Trip'].unique()

#### Mencocokan Stops Seq dan Data DO sebelum beneran dimerge

<b>Open previously used Stops Sequence</b>

In [166]:
ss0 = pd.read_excel('Agustus-8/Stop Seq 8 Agustus_Updated.xlsx')

In [167]:
dftrip.loc[dftrip['Rute Real']=='LATUMETEN\xa0-\xa0PGC 1']

,No Do,No Bus,Kode Rute Real,Tanggal Do,TimeStart,TimeEnd,Rute Real,TTMinutes
No,,,,,,,,


In [168]:
dftrip.loc[dftrip['Rute Real']=='LATUMETEN\xa0-\xa0PGC 1', 'Rute Real'] = 'LATUMETEN-PGC 1'
dftrip.loc[dftrip['Rute Real']=='MONAS-KALIDERES', 'Rute Real'] = 'MONAS-KALIDERES VIA VETERAN'
dftrip.loc[dftrip['Rute Real']=='PASAR BARU-PASAR KEBAYORAN LAMA-PASAR BARU', 'Rute Real'] = 'PASAR BARU-PASAR KEBAYORAN LAMA-PASAR BARU VIA TOMANG'

In [248]:
# Wild trip dari DO Manual
dftrip.loc[dftrip['Rute Real']=='TANAH ABANG-ST. GONDANGDIA-TANAH ABANG', 'Rute Real'] = 'TANAH ABANG-STASIUN GONDANGDIA-TANAH ABANG'
dftrip.loc[dftrip['Rute Real']=='MONAS-KALIDERES', 'Rute Real'] = 'MONAS-KALIDERES VIA VETERAN'


In [250]:
dftrip[dftrip['Rute Real']=='JELAMBAR-KOTA-JELAMBAR']

,No Do,No Bus,Kode Rute Real,Tanggal Do,TimeStart,TimeEnd,Rute Real,TTMinutes
No,,,,,,,,
6469,202309221306501331,SAF104,3H,2023-09-22,2023-09-22 13:07:00,2023-09-22 15:03:00,JELAMBAR-KOTA-JELAMBAR,116.0
6604,202309221351531247,SAF119,3H,2023-09-22,2023-09-22 13:54:00,2023-09-22 15:32:00,JELAMBAR-KOTA-JELAMBAR,98.0
8016,202309220526481119,TJ0411,3H,2023-09-22,2023-09-22 05:28:00,2023-09-22 07:02:00,JELAMBAR-KOTA-JELAMBAR,94.0
8017,202309220702091010,TJ0411,3H,2023-09-22,2023-09-22 07:02:00,2023-09-22 08:50:00,JELAMBAR-KOTA-JELAMBAR,108.0
8018,202309221531101254,TJ0411,3H,2023-09-22,2023-09-22 15:31:00,2023-09-22 21:19:00,JELAMBAR-KOTA-JELAMBAR,348.0
8024,202309220502231145,TJ0416,3H,2023-09-22,2023-09-22 05:53:00,2023-09-22 10:48:00,JELAMBAR-KOTA-JELAMBAR,295.0
8025,202309221413061498,TJ0416,3H,2023-09-22,2023-09-22 14:14:00,2023-09-22 22:32:00,JELAMBAR-KOTA-JELAMBAR,498.0
8027,202309220552311109,TJ0417,3H,2023-09-22,2023-09-22 05:52:00,2023-09-22 10:49:00,JELAMBAR-KOTA-JELAMBAR,297.0
8028,202309221052321002,TJ0417,3H,2023-09-22,2023-09-22 10:54:00,2023-09-22 16:59:00,JELAMBAR-KOTA-JELAMBAR,365.0


In [178]:
triptransit = dftrip.groupby(['Kode Rute Real', 'Rute Real']).count()[['Tanggal Do']].reset_index().drop('Tanggal Do', axis=1)

In [179]:
triptransit['Rute Real'] = triptransit['Rute Real'].apply(lambda x: x.strip())

In [206]:
former_trips = pd.read_csv('Agustus-8/DO 8 Agustus_Processed.csv', dtype={'No Do': 'str'})
former_trips.head(1)
former_trips = former_trips.groupby(['Kode Rute Real', 'Rute Real', 'Nama Trip', 'Kode Trip']).count()[['TTMinutes']].reset_index()

In [235]:
former_trips[former_trips['Kode Rute Real']=='10']

,Kode Rute Real,Rute Real,Nama Trip,Kode Trip,TTMinutes
7,10,PGC 2-SUNTER KELAPA GADING-PGC 2,PGC 2-SUNTER KELAPA GADING-PGC 2,10-P22,10
8,10,PGC 2-TANJUNG PRIOK,PGC 2-TANJUNG PRIOK,10-R02,157
9,10,PGC 2-WALIKOTA JAKUT-PGC 2,PGC 2-WALIKOTA JAKUT-PGC 2,10-P23,4
10,10,TANJUNG PRIOK-PGC 2,TANJUNG PRIOK-PGC 2,10-R01,148
11,10,TANJUNG PRIOK-PGC 2-TANJUNG PRIOK,TANJUNG PRIOK-PGC 2-TANJUNG PRIOK,10-L01,1


In [231]:
merge1 = triptransit.merge(former_trips, how='left', left_on=['Kode Rute Real', 'Rute Real'], 
                           right_on=['Kode Rute Real', 'Rute Real'])

unmatchedtrip = merge1[merge1['Nama Trip'].isna()][['Kode Rute Real', 'Rute Real']]
merge1 = merge1[merge1['Nama Trip'].isna()==False]

In [234]:
unmatchedtrip

45

In [245]:
matched2 = unmatchedtrip.merge(ss0, how='left', left_on=['Kode Rute Real', 'Rute Real'], 
                    right_on=['Kode Rute', 'Nama Trip'])

len(matched2[matched2['Nama Trip'].isna()==False]['Rute Real'].unique())
len(matched2[matched2['Nama Trip'].isna()==False]['Kode Trip'].unique())

# matched2.to_excel('matched2_22sep.xlsx')

In [244]:
len(matched2[matched2['Nama Trip'].isna()]['Rute Real'].unique())

16

In [163]:
# ss = stopseq0.groupby(['Kode Rute', 'Nama Trip', 'Kode Trip', 'STATUS OPERASI TRIP TRANSIT']).count().reset_index()[['Kode Rute', 'Nama Trip', 'Kode Trip', 'STATUS OPERASI TRIP TRANSIT']]

# merge1 = triptransit.merge(ss, how='left', left_on=['Kode Rute Real', 'Rute Real'], right_on=['Kode Rute', 'Nama Trip'])

In [165]:
# stopseq0[stopseq0['Nama Trip']=='PASAR BARU-PASAR KEBAYORAN LAMA-PASAR BARU VIA TOMANG']
# stopseq0[stopseq0['Kode Trip']=='6A-L02']

In [181]:
# triptransit

In [233]:
# merge1.to_excel('Trip Operasi 8 Agustus_Try 2.xlsx')

In [173]:
# Ada beberapa trip yang nama trip nya masih nama trip lama, padahal lintasannya sudah baru. Maka nama tripnya diganti saja 

dftrip.loc[dftrip['Rute Real']=='JUANDA-PGC', 'Rute Real']='JUANDA-PGC VIA BUDI UTOMO'

# dftrip[dftrip['Rute Real'].str.contains('PGC')]

In [177]:
dftrip.head(1)

,No Do,No Bus,Kode Rute Real,Tanggal Do,TimeStart,TimeEnd,Rute Real,TTMinutes
No,,,,,,,,
0,202309220558051375,BHL220508,7D,2023-09-22,2023-09-22 05:59:00,2023-09-22 07:59:00,TMII - TEGAL PARANG - TMII VIA CAWANG SUTOYO,120.0


In [196]:
# dftrip['Nama Trip'] = dftrip['Nama Trip'].apply(lambda x: x.replace(' -', '-'))
# dftrip['Nama Trip'] = dftrip['Nama Trip'].apply(lambda x: x.replace('- ', '-'))

dftrip['Rute Real'] = dftrip['Rute Real'].apply(lambda x: x.replace(' -', '-'))
dftrip['Rute Real'] =dftrip['Rute Real'].apply(lambda x: x.replace('- ', '-'))

triptransit['Rute Real'] = triptransit['Rute Real'].apply(lambda x: x.replace(' -', '-'))
triptransit['Rute Real'] = triptransit['Rute Real'].apply(lambda x: x.replace('- ', '-'))



In [190]:
ss0.head(1)

,ID,Kode Rute,Nama Rute,Kode Trip,Nama Trip,Stop ID,Nama Perhentian,Koordinat,Latitude,Longitude,Urutan,Kode PP,Keterangan,Tipe Perhentian,Kategori Rute,Utama / Non,STATUS OPERASI RUTE,STATUS OPERASI HALTE,STATUS OPERASI TRIP GTFS,STATUS OPERASI TRIP TRANSIT,Unnamed: 20,split,Looping
0,1,1,BLOK M - KOTA,1-P01,KOTA-HARMONI,H00026C,KOTA,-6.13782,106.81387,-6.13782,106.81387,1,1,PERGI,Halte,BRT,Non-Reguler,Operasi,Tidak Operasi,Tidak Beroperasi,Tidak Beroperasi,2,One Trip


In [197]:
triptransit.head(1)

,Kode Rute Real,Rute Real
0,1,BLOK M-KALI BESAR BARAT


In [198]:
merge1 = triptransit.merge(ss0, left_on='Rute Real', right_on='Nama Trip', how='left')
merge1[merge1['Nama Perhentian'].isna()]['Rute Real'].unique()

array(['TENDEAN-HALTE JOOR-KUNINGAN',
       'TANAH ABANG-ST. GONDANGDIA-TANAH ABANG', 'JELAMBAR-KOTA-JELAMBAR',
       'KOTA-JELAMBAR', 'PGC 1-JUANDA',
       'LEBAK BULUS-SETIABUDI UTARA-LEBAK BULUS',
       'TMII-TEGAL PARANG-TMII', 'TMII-TEGAL PARANG TMII',
       'PGC 1 -LATUMETEN', 'BULAK KAPAL-BNN-BULAK KAPAL',
       'KALIDERES-PURI KEMBANGAN', 'JIS-JUANDA-JIS',
       'TENDEAN-PURI BETA 2', 'PULO GADUNG 2-MONAS'], dtype=object)

#### Merge Data DO (dftrip) dengan Kode Trip

In [238]:
merge3=dftrip.merge(merge2, left_on=['Rute Real', 'Kode Rute Real'], right_on=['Rute Real', 'Kode Rute Real'], how='left')
len(merge3)

10705

In [239]:
merge3[merge3['Kode Trip'].isna()]['Rute Real'].unique()

array([], dtype=object)

Memastikan file merge2 (file yg sudah mencocokan nama rute real dan kode trip, benar bisa dimerge dengan stops sequence)

In [240]:
merge2_ss= merge2[['Rute Real', 'Nama Trip', 'Kode Trip']]

a1 = stopseq0.merge(merge2_ss, on=['Nama Trip', 'Kode Trip'], how='right')
a1[a1['ID'].isna()]

,ID,Kode Rute,Nama Rute,Kode Trip,Nama Trip,Stop ID,Nama Perhentian,Koordinat,Latitude,Longitude,Urutan,Kode PP,Keterangan,Tipe Perhentian,Kategori Rute,Utama / Non,STATUS OPERASI RUTE,STATUS OPERASI HALTE,STATUS OPERASI TRIP GTFS,STATUS OPERASI TRIP TRANSIT,Unnamed: 20,split,Looping,Rute Real
2926,NaN,NaN,NaN,5C-R10,JUANDA-PGC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JUANDA-PGC


In [140]:
# stopseq0[stopseq0['Nama Trip']=='PORIS PLAWAD-SLIPI KEMANGGISAN-PORIS PLAWAD']['Kode Trip'].unique()
# stopseq0[stopseq0['Kode Trip']=='T11-L02']

In [142]:
# dftrip

#### Output Data DO yang sudah bersih dan cocok: 

In [242]:
len(dftrip), len(merge3)

(10705, 10705)

In [241]:
merge3.drop(['STATUS OPERASI TRIP TRANSIT', 'Unnamed: 6'], axis=1, inplace=True)

merge3.head(2)


,No Do,No Bus,Kode Rute Real,Tanggal Do,TimeStart,TimeEnd,Rute Real,TTMinutes,Kode Rute,Nama Trip,Kode Trip
0,202308080603431331,BHL220508,7D,2023-08-08,2023-08-08 06:05:00,2023-08-08 07:58:00,TMII-TEGAL PARANG-TMII VIA CAWANG SUTOYO,113.0,7D,TMII-TEGAL PARANG-TMII VIA CAWANG SUTOYO,7D-L06
1,202308080758181741,BHL220508,7D,2023-08-08,2023-08-08 08:07:00,2023-08-08 10:30:00,TMII-TEGAL PARANG-TMII VIA CAWANG SUTOYO,143.0,7D,TMII-TEGAL PARANG-TMII VIA CAWANG SUTOYO,7D-L06


In [244]:
merge3.to_csv('Agustus-8/DO 8 Agustus_Processed.CSV', index=False)
# stopseq0.to_excel('Agustus-8/Stop Seq 8 Agustus_Updated.xlsx', index=False)
merge2.to_excel('Agustus-8/Kode Trip and Rute Real Matched_8 Agustus.xlsx', index=False)

In [154]:
merge3['TTMinutes'].min()

18.0

In [251]:
stopseq0

,ID,Kode Rute,Nama Rute,Kode Trip,Nama Trip,Stop ID,Nama Perhentian,Koordinat,Latitude,Longitude,Urutan,Kode PP,Keterangan,Tipe Perhentian,Kategori Rute,Utama / Non,STATUS OPERASI RUTE,STATUS OPERASI HALTE,STATUS OPERASI TRIP GTFS,STATUS OPERASI TRIP TRANSIT,Unnamed: 20,split,Looping
0,1,1,BLOK M - KOTA,1-P01,KOTA-HARMONI,H00026C,KOTA,-6.137820,106.813870,-6.137820,106.813870,1,1,PERGI,Halte,BRT,Non-Reguler,Operasi,Tidak Operasi,Tidak Beroperasi,Tidak Beroperasi,2,One Trip
1,2,1,BLOK M - KOTA,1-P01,KOTA-HARMONI,H00068P,GLODOK,-6.144500,106.815450,-6.144500,106.815450,2,1,PERGI,Halte,BRT,Non-Reguler,Operasi,Operasi,Tidak Beroperasi,Tidak Beroperasi,2,One Trip
2,3,1,BLOK M - KOTA,1-P01,KOTA-HARMONI,H00133P,OLIMO,-6.149300,106.816690,-6.149300,106.816690,3,1,PERGI,Halte,BRT,Non-Reguler,Operasi,Operasi,Tidak Beroperasi,Tidak Beroperasi,2,One Trip
3,4,1,BLOK M - KOTA,1-P01,KOTA-HARMONI,H00124P,MANGGA BESAR,-6.152170,106.817360,-6.152170,106.817360,4,1,PERGI,Halte,BRT,Non-Reguler,Operasi,Operasi,Tidak Beroperasi,Tidak Beroperasi,2,One Trip
4,5,1,BLOK M - KOTA,1-P01,KOTA-HARMONI,H00208P,SAWAH BESAR,-6.160410,106.819220,-6.160410,106.819220,5,1,PERGI,Halte,BRT,Non-Reguler,Operasi,Operasi,Tidak Beroperasi,Tidak Beroperasi,2,One Trip
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38379,12698,6A,RAGUNAN - MH THAMRIN VIA KUNINGAN,6A-L02,RAGUNAN-MH THAMRIN-RAGUNAN VIA KUNINGAN,H00158P,PEJATEN,-6.278422,106.829762,-6.278422,106.829762,34,2,PULANG,Halte,BRT,Reguler,Operasi,Operasi,Beroperasi,Beroperasi,3,Looping
38380,12699,6A,RAGUNAN - MH THAMRIN VIA KUNINGAN,6A-L02,RAGUNAN-MH THAMRIN-RAGUNAN VIA KUNINGAN,H00081P,JATI PADANG,-6.285349,106.826329,-6.285349,106.826329,35,2,PULANG,Halte,BRT,Reguler,Operasi,Operasi,Beroperasi,Beroperasi,3,Looping
38381,12700,6A,RAGUNAN - MH THAMRIN VIA KUNINGAN,6A-L02,RAGUNAN-MH THAMRIN-RAGUNAN VIA KUNINGAN,H00221P,SMK 57,-6.290979,106.823689,-6.290979,106.823689,36,2,PULANG,Halte,BRT,Reguler,Operasi,Operasi,Beroperasi,Beroperasi,3,Looping
38382,12701,6A,RAGUNAN - MH THAMRIN VIA KUNINGAN,6A-L02,RAGUNAN-MH THAMRIN-RAGUNAN VIA KUNINGAN,H00044P,DEPARTEMEN PERTANIAN,-6.294671,106.822135,-6.294671,106.822135,37,2,PULANG,Halte,BRT,Reguler,Operasi,Operasi,Beroperasi,Beroperasi,3,Looping


### 3. Matching Trip ID

1. Membersihkan dftrips dan membuat daftartrip

In [44]:
### Setelah dimatch, kembali ke sini untuk merubah beberapa nama trip agar sesuai dengan nama trip yg ada di database

zerotrip2 = pd.read_excel('Trip Tanpa Kode Trip 2.xlsx')

In [46]:
zerotrip2

,Unnamed: 0,Kode Rute Real,Rute Real,New Rute Real,Jumlah,Trip ID,Keterangan GIS
0,13,5C,PGC - JUANDA - PGC VIA BUDI UTOMO,PGC - JUANDA - PGC VIA BUDI UTOMO,457,5C-L07,NaN
1,9,3,KALIDERES - BUNDARAN HI - KALIDERES,KALIDERES - BUNDARAN HI - KALIDERES,214,3-L01,FILE GIS NEW
2,19,5M,KAMPUNG MELAYU - TANAH ABANG - KAMPUNG MELAYU ...,KAMPUNG MELAYU - TANAH ABANG - KAMPUNG MELAYU ...,198,5M-L01,GANTI FILE GIS
3,38,JIS3,JIS - MANGGA BESAR - JIS,JIS - MANGGA BESAR - JIS,135,JIS3-L02,FILE GIS NEW
4,39,M3,KALIDERES - BUNDARAN HI - KALIDERES VIA VETERAN,KALIDERES - BUNDARAN HI - KALIDERES VIA VETERAN,107,M3-L02,FILE GIS NEW
5,8,3,KALIDERES - BALAI KOTA VIA CIDENG,KALIDERES - BALAI KOTA VIA CIDENG,46,3-R09,FILE GIS TIDAK ADA
6,17,5M,KAMPUNG MELAYU - EXPLORER TANAH ABANG - KAMPUN...,KAMPUNG MELAYU - EXPLORER TANAH ABANG - KAMPUN...,39,5M-L03,NaN
7,24,6Q,EPICENTRUM RAYA - DUKU ATAS VIA CASABLANCA - E...,EPICENTRUM - CASABLANCA - EPICENTRUM,24,6Q-L02,NaN
8,15,5C,PGC - JUANDA VIA KWITANG,PGC - JUANDA VIA KWITANG,12,5C-R09,FILE GIS TIDAK ADA
9,30,8M,TANAH ABANG - S PARMAN - TANAH ABANG,TANAH ABANG - S. PARMAN - TANAH ABANG,12,8M-L01,NaN


In [92]:
zerotrip2= zerotrip2[['Rute Real', 'New Rute Real']]
zerotrip2.head(3)

,Rute Real,New Rute Real
0,PGC - JUANDA - PGC VIA BUDI UTOMO,PGC - JUANDA - PGC VIA BUDI UTOMO
1,KALIDERES - BUNDARAN HI - KALIDERES,KALIDERES - BUNDARAN HI - KALIDERES
2,KAMPUNG MELAYU - TANAH ABANG - KAMPUNG MELAYU ...,KAMPUNG MELAYU - TANAH ABANG - KAMPUNG MELAYU ...


In [206]:
for i in range(len(zerotrip2)):
    r0 = zerotrip2['Rute Real'].iloc[i]
    r1 = zerotrip2['New Rute Real'].iloc[i]
    dftrip.loc[dftrip['Rute Real']==r0, 'Rute Real'] = r1
    
dftrip.loc[dftrip['Kode Rute Real'] == 'JELAMBAR - BSD - JELAMBA', 'Kode Rute Real']= 'S11'
dftrip.loc[dftrip['Kode Rute Real'] == 'JELAMBAR - T.BS', 'Kode Rute Real']= 'S11'
dftrip.loc[dftrip['Rute Real']=='KALIDERES - BUNDARAN HI - KALIDERES', 'Rute Real'] = 'KALIDERES - BUNDARAN HI - KALIDERES VIA VETERAN'
    
# dftrip.loc[dftrip['Rute Real']=='TANAH ABANG - S PARMAN - TANAH ABANG']

In [93]:
## Start the actual matching process

In [207]:
daftartrip = pd.read_excel('../Other Scripts/Nama Trip Aktif_20230308.xlsx')
daftartrip = daftartrip.drop('Nama Trip 2', axis=1)

In [208]:
dftrip['Kode Rute Real'] = dftrip['Kode Rute Real'].apply(lambda x: str(x))
daftartrip['Kode Rute'] = daftartrip['Kode Rute'].apply(lambda x: str(x))

In [104]:
tripunique = dftrip.drop_duplicates(subset=['Kode Rute Real', 'Rute Real'])
tripunique = tripunique[['Kode Rute Real', 'Rute Real']]
tripunique.head(2)

,Kode Rute Real,Rute Real
2,6,RAGUNAN - HALIMUN
3,6,HALIMUN - RAGUNAN


#### Membersihkan Merge dengan Daftar Trip Aktif dan Basis Data dan Sumber Lainnya (Manual dari Transit)

In [115]:
tripmerge1 = tripunique.merge(daftartrip, left_on=['Kode Rute Real', 'Rute Real'], right_on=['Kode Rute', 'Nama Trip'], how='left')
print(tripmerge1['Nama Trip'].isna().sum())
tripmerge1

,Kode Rute Real,Rute Real,No,Kategori Rute,Kode Rute,Kode Trip,Nama Trip
0,6,RAGUNAN - HALIMUN,73.0,BRT,6,6-P01,RAGUNAN - HALIMUN
1,6,HALIMUN - RAGUNAN,74.0,BRT,6,6-P02,HALIMUN - RAGUNAN
2,6B,RAGUNAN - MH THAMRIN VIA SEMANGGI,79.0,BRT,6B,6B-R03,RAGUNAN - MH THAMRIN VIA SEMANGGI
3,6B,MH THAMRIN - RAGUNAN VIA SEMANGGI,80.0,BRT,6B,6B-R04,MH THAMRIN - RAGUNAN VIA SEMANGGI
4,6B,RAGUNAN - MH THAMRIN - RAGUNAN VIA SEMANGGI,81.0,BRT,6B,6B-L02,RAGUNAN - MH THAMRIN - RAGUNAN VIA SEMANGGI
...,...,...,...,...,...,...,...
223,3,BUNDARAN HI - KALIDERES VIA VETERAN,45.0,BRT,3,3-R04,BUNDARAN HI - KALIDERES VIA VETERAN
224,M8,PASAR BARU - LEBAK BULUS,155.0,BRT,M8,M8-R04,PASAR BARU - LEBAK BULUS
225,5C,PGC - JUANDA - PGC,63.0,BRT,5C,5C-L05,PGC - JUANDA - PGC
226,8M,TANAH ABANG - S. PARMAN,531.0,Non-BRT,8M,8M-R02,TANAH ABANG - S. PARMAN


In [106]:
tripzero = tripmerge1[tripmerge1['Nama Trip'].isna()]
listtripzero = list(tripzero['Rute Real'])
tripzero = dftrip[dftrip['Rute Real'].isin(listtripzero)]
tripzero = tripzero.groupby(['Kode Rute Real', 'Rute Real']).count()[['TimeStart']].reset_index()
#tripzero.to_excel('Trip Tanpa Kode Trip.xlsx')

In [107]:
tripbasdat = pd.read_csv('../../Basis Data Alat Produksi/20230428_Basis Data Alat Produksi - Daftar Trip.csv', delimiter=';')

In [108]:
tripbasdat.head(1)

,Trip ID,Kode Rute,Tipe Trip,Nama Trip,trip_headsign,trip_headsign_2,KM Baku/GIS,Jenis KM,Stop Sequence,File GIS,Status RKA 2021,Urutan Trip,Status Operasi,Kode Hari Operasi
0,1-R02,1,Reguler,KOTA - BLOK M,KOTA - BLOK M,NaN,"13,436.40",BAKU,Ada,1-R02 Kota - Blok M.geojson,Beroperasi,2,Tidak Beroperasi,SH


In [110]:
tripmerge2 = tripzero.merge(tripbasdat, left_on=['Rute Real', 'Kode Rute Real'], right_on=['Nama Trip', 'Kode Rute'], how='left')
# tripmerge2[tripmerge2['Trip ID'].isna()==False].sort_values(by=['Rute Real', 'Trip ID'])
listtripzero2 = list(tripmerge2[tripmerge2['Trip ID'].isna()]['Rute Real'])

In [111]:
tripzero2 = tripzero[tripzero['Rute Real'].isin(listtripzero2)].sort_values(by='TimeStart', ascending=False)
#tripzero2.to_excel('Trip Tanpa Kode Trip 3.xlsx')

In [130]:
tripzero3 = pd.read_excel('Trip Tanpa Kode Trip 2.xlsx')
tripzero3 = tripzero3[['Kode Rute Real', 'Rute Real', 'Trip ID']]
tripzero3.rename(columns={'Rute Real' : 'Nama Trip'}, inplace=True)
tripzero3.head(1)

,Kode Rute Real,Nama Trip,Trip ID
0,5C,PGC - JUANDA - PGC VIA BUDI UTOMO,5C-L07


In [128]:
tripmerge1 = tripmerge1[['Kode Rute Real', 'Rute Real', 'Kode Trip']]
tripmerge1.rename(columns={'Rute Real': 'Nama Trip', 'Kode Trip':'Trip ID'}, inplace=True)
tripmerge1.head(1)

,Kode Rute Real,Nama Trip,Trip ID
0,6,RAGUNAN - HALIMUN,6-P01


In [124]:
tripmerge2 = tripmerge2[['Kode Rute Real', 'Nama Trip', 'Trip ID']]
tripmerge2 = tripmerge2[tripmerge2['Trip ID'].isna()==False]
tripmerge2.head(1)

,Kode Rute Real,Nama Trip,Trip ID
0,1,BUNDARAN SENAYAN - HARMONI - BUNDARAN SENAYAN,1-P29


In [209]:
# trips = pd.concat([tripmerge1, tripmerge2, tripzero3])
# len(trips)

trips = pd.read_excel('20230428_KODE TRIP DAN NAMA TRIP YANG DIGUNAKAN.xlsx')
trips = trips[['Kode Rute Real', 'Nama Trip', 'Trip ID']]
trips
#trips.to_excel('20230428_KODE TRIP DAN NAMA TRIP YANG DIGUNAKAN.xlsx')

,Kode Rute Real,Nama Trip,Trip ID
0,5,ANCOL - MATRAMAN BARU,5-R04
1,5,ANCOL - MATRAMAN BARU - ANCOL,5-L03
2,M5,ANCOL - PGC 2,M5-R02
3,3,BALAI KOTA - KALIDERES VIA CIDENG,3-R10
4,1A,BALAI KOTA - PANTAI MAJU,1A-R04
...,...,...,...
221,S11,TERMINAL BSD - JELAMBAR - TERMINAL BSD,S11-L01
222,6H,TERMINAL SENEN - LEBAK BULUS,6H-R03
223,6H,TERMINAL SENEN - LEBAK BULUS - TERMINAL SENEN,6H-L02
224,4C,TU GAS - BUNDARAN SENAYAN,4C-R01


#### 3. Menggabungkan Trip ID dengan dftrip

In [199]:
###### for this batch


# dftrip.loc[dftrip['Nama Trip']=='KALIDERES - BUNDARAN HI - KALIDERES VIA VETERAN', 'Trip ID']='M3-L02'

In [189]:
# dftrip

In [214]:
dftrip['Kode Rute Real'] = dftrip['Kode Rute Real'].apply(lambda x: str(x).strip())
trips['Kode Rute Real'] = trips['Kode Rute Real'].apply(lambda x: str(x).strip())
dftrip.rename(columns={'Rute Real': 'Nama Trip'}, inplace=True)

#this batch only

dftrip.loc[dftrip['Nama Trip']=='JELAMBAR - T.BSD' , 'Nama Trip'] = 'JELAMBAR - TERMINAL BSD'

In [216]:
dft1 = dftrip.merge(trips, how='left', on=['Kode Rute Real', 'Nama Trip'])
print(dft1[dft1['Trip ID'].isna()]['Nama Trip'].unique())
dft1 = dft1[~dft1['Trip ID'].isna()]

['(9)']


In [218]:
dft1.to_csv('20230428_Processed and Matched Trips.csv')

### 5. Menggabungkan trips dengan nama file goejson